In [15]:
import pymupdf

file_path = r"C:\Users\sekhar\Downloads\Fillable-Form_CIOMS-to-E2B.pdf"

doc = pymupdf.open(file_path)
widgets_list = []
radio_button_group = {}
rect_radio_buttons = []
json_data = {}
for page in doc:
    widgets = page.widgets()
    if widgets:
        l = []
        
        for  widget in widgets:

            # if widget.field_type
            if widget.field_type_string == 'RadioButton':
                # if widget.field_name 
                print(f"Radio button group before {radio_button_group}")
                same_radio_field_list = radio_button_group.get(widget.field_name, [])
                print(f"getting the same radio field list before {same_radio_field_list}")
                # import ipdb;ipdb.set_trace(context=5)
                print(f"field name original : {widget.field_name}")
                value = f"{widget.field_name}____{len(same_radio_field_list)+1}"
                print(f"value genrating {value}")
                
                l.append(value)
                same_radio_field_list.append(value)
                radio_button_group[widget.field_name] = same_radio_field_list

                print(f"Radio button group affter {radio_button_group}")
                print(f"getting the same radio field list after {same_radio_field_list}")

                json_data[value] = value
                print(f"text with in the radio button bbox :== {page.get_textbox(widget.rect)}")
                page.add_highlight_annot(widget.rect)
                
                rect_radio_buttons.append(widget.rect)
            else:
                l.append(widget.field_name)
                json_data[widget.field_name] = widget.field_name
            print(f"widget type {widget.field_type}")
            print(f"widget field type string {widget.field_type_string}")
            print(f"widget label {widget.field_label}")
            print(f"widget name {widget.field_name}")
            print(f"widget value {widget.field_value}")
            print(f"widget choice values {widget.choice_values}")
            print(f"widget display {widget.field_display}")
            print('*'*50)
        # widgets_list.extend([widget.field_name for widget in widgets])
        widgets_list.extend(l)

        import json
        with open('test_josn_data.json', 'w') as fp:
            json.dump(json_data, fp, indent=4)

doc.save('testing_highlighted_pdf.pdf')
rect_radio_buttons
dir(doc[0])
count = 0
for page in doc:
    print(f"In {count + 1} page")
    widgets = page.widgets()
    if widgets:
        
        radio_b = 0
        for radio in rect_radio_buttons:
            print(f"radio button {radio_b + 1}")
            print(page.get_textbox(radio))
            radio_b += 1
    count += 1

widget type 7
widget field type string Text
widget label SUSPECT ADVERSE REACTION REPORT
widget name SUSPECT ADVERSE REACTION REPORT
widget value 
widget choice values None
widget display 0
**************************************************
widget type 7
widget field type string Text
widget label 1. PATIENT INITIALS (first, last)
widget name 1 PATIENT INITIALS first last
widget value 
widget choice values None
widget display 0
**************************************************
widget type 7
widget field type string Text
widget label 1a. COUNTRY
widget name 1a COUNTRY
widget value 
widget choice values None
widget display 0
**************************************************
widget type 7
widget field type string Text
widget label None
widget name 2_DateOfBirth
widget value 
widget choice values None
widget display 0
**************************************************
widget type 7
widget field type string Text
widget label 2a. AGE Years
widget name 2a AGE Years
widget value 
widget choic

In [21]:
import fitz  # PyMuPDF
import json

file_path = r"C:\Users\sekhar\Downloads\Fillable-Form_CIOMS-to-E2B.pdf"

doc = fitz.open(file_path)
widgets_list = []
radio_button_group = {}
rect_radio_buttons = []
json_data = {}
label_map = {}  # Dictionary to store labels for each radio button group

for page in doc:
    widgets = page.widgets()
    if widgets:
        l = []

        widget_radio_num = 0
        for widget in widgets:
            if widget.field_type_string == 'RadioButton':
                print(f"Radio button group before {radio_button_group}")
                same_radio_field_list = radio_button_group.get(widget.field_name, [])
                print(f"Getting the same radio field list before {same_radio_field_list}")
                print(f"Field name original: {widget.field_name}")
                value = f"{widget.field_name}____{len(same_radio_field_list)+1}"
                print(f"Value generating: {value}")

                l.append(value)
                same_radio_field_list.append(value)
                radio_button_group[widget.field_name] = same_radio_field_list

                print(f"Radio button group after {radio_button_group}")
                print(f"Getting the same radio field list after {same_radio_field_list}")

                bounding_box = widget.rect  # (x0, y0, x1, y1)

                # Define the amount by which to increase the bounding box horizontally
                increase_amount = 20  # Increase by 20 units on both sides

                # Adjust the bounding box
                new_bounding_box = pymupdf.Rect(bounding_box.x0 - increase_amount, bounding_box.y0,
                                            bounding_box.x1 + increase_amount, bounding_box.y1)
                
                pix = page.get_pixmap(clip=new_bounding_box)
                image_path = f"page_{widget_radio_num + 1}_bbox_image.png"
                pix.save(image_path)

                import pytesseract
                from PIL import Image

                with Image.open(image_path) as img:
                    text = pytesseract.image_to_string(img)
                    print(f"Text extracted from bounding box on radio button {widget_radio_num}: {text}")
                    print(text)
                    print('-' * 50)


                widget_radio_num += 1

                # Retrieve the text surrounding the widget's bounding box
                surrounding_text = page.get_textbox(new_bounding_box)
                print(f"Text within the radio button bbox: {surrounding_text}")

                # Highlight the widget for visual debugging
                page.add_highlight_annot(new_bounding_box)

                # Store the bounding box and text information
                rect_radio_buttons.append({
                    "rect": widget.rect,
                    "text": surrounding_text,
                    "name": value
                })

                # Store the label text in a map
                if widget.field_name not in label_map:
                    label_map[widget.field_name] = []
                label_map[widget.field_name].append(surrounding_text.strip())

                json_data[value] = {
                    "name": value,
                    "bounding_box": list(widget.rect),
                    "text": surrounding_text.strip()
                }
            else:
                l.append(widget.field_name)
                json_data[widget.field_name] = {
                    "name": widget.field_name,
                    "bounding_box": list(widget.rect),
                    "text": page.get_textbox(widget.rect).strip()
                }
                # page.add_highlight_annot(widget.rect)

            print(f"Widget type: {widget.field_type}")
            print(f"Widget field type string: {widget.field_type_string}")
            print(f"Widget label: {widget.field_label}")
            print(f"Widget name: {widget.field_name}")
            print(f"Widget value: {widget.field_value}")
            print(f"Widget choice values: {widget.choice_values}")
            print(f"Widget display: {widget.field_display}")
            print(f"Widget bounding box: {widget.rect}")
            print('*' * 50)

        widgets_list.extend(l)

# Save the enhanced PDF with highlighted widgets
doc.save('testing_highlighted_pdf.pdf')

# Save the extracted data to a JSON file
with open('test_json_data.json', 'w') as fp:
    json.dump(json_data, fp, indent=4)

# Print the label map for debugging
print(json.dumps(label_map, indent=4))


Widget type: 7
Widget field type string: Text
Widget label: SUSPECT ADVERSE REACTION REPORT
Widget name: SUSPECT ADVERSE REACTION REPORT
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(301.20001220703125, 91.85198974609375, 532.7999877929688, 114.4119873046875)
**************************************************
Widget type: 7
Widget field type string: Text
Widget label: 1. PATIENT INITIALS (first, last)
Widget name: 1 PATIENT INITIALS first last
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(60.720001220703125, 181.3709716796875, 144.47999572753906, 197.093994140625)
**************************************************
Widget type: 7
Widget field type string: Text
Widget label: 1a. COUNTRY
Widget name: 1a COUNTRY
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(146.63999938964844, 181.3709716796875, 219.1199951171875, 197.093994140625)
****************************************

In [65]:
import fitz  # PyMuPDF
import json

file_path = r"C:\Users\sekhar\Downloads\Fillable-Form_CIOMS-to-E2B.pdf"

doc = fitz.open(file_path)
widgets_list = []
radio_button_group = {}
rect_radio_buttons = []
json_data = {}
label_map = {}  # Dictionary to store labels for each radio button group

for page in doc:
    widgets = page.widgets()
    if widgets:
        l = []

        widget_radio_num = 0
        for widget in widgets:
            if widget.field_type_string == 'RadioButton':
                
                same_radio_field_list = radio_button_group.get(widget.field_name, [])
                
                value = f"{widget.field_name}____{len(same_radio_field_list)+1}"
                

                l.append(value)
                same_radio_field_list.append(value)
                radio_button_group[widget.field_name] = same_radio_field_list


                bounding_box = widget.rect  # (x0, y0, x1, y1)

                # Define the amount by which to increase the bounding box horizontally
                increase_amount = 20  # Increase by 20 units on both sides

                # Adjust the bounding box
                new_bounding_box = pymupdf.Rect(bounding_box.x0 + 14, bounding_box.y0 + 3,
                                            bounding_box.x1 + increase_amount*2, bounding_box.y1)
                
                zoom_x = 9.0
                zoom_y = 8.0
                mat = pymupdf.Matrix(zoom_x, zoom_y)
                pix = page.get_pixmap(matrix=mat, clip=new_bounding_box)
                image_path = f"page_{widget_radio_num + 1}_bbox_image.png"
                pix.save(image_path)

                import pytesseract
                from PIL import Image

                with Image.open(image_path) as img:
                    text = pytesseract.image_to_string(img)
                    print(f"Text extracted from bounding box on radio button {widget_radio_num}: {text}")
                    print(text)
                    print('-' * 50)


                widget_radio_num += 1

                # Retrieve the text surrounding the widget's bounding box
                surrounding_text = page.get_textbox(new_bounding_box)
                print(f"Text within the radio button bbox: {surrounding_text}")

                # Highlight the widget for visual debugging
                page.add_highlight_annot(new_bounding_box)

                # Store the bounding box and text information
                rect_radio_buttons.append({
                    "rect": widget.rect,
                    "text": surrounding_text,
                    "name": value
                })

                # Store the label text in a map
                if widget.field_name not in label_map:
                    label_map[widget.field_name] = []
                label_map[widget.field_name].append(surrounding_text.strip())

                json_data[value] = {
                    "name": value,
                    "bounding_box": list(widget.rect),
                    "text": surrounding_text.strip()
                }
            else:
                l.append(widget.field_name)
                json_data[widget.field_name] = {
                    "name": widget.field_name,
                    "bounding_box": list(widget.rect),
                    "text": page.get_textbox(widget.rect).strip()
                }
                # page.add_highlight_annot(widget.rect)

            print(f"Widget type: {widget.field_type}")
            print(f"Widget field type string: {widget.field_type_string}")
            print(f"Widget label: {widget.field_label}")
            print(f"Widget name: {widget.field_name}")
            print(f"Widget value: {widget.field_value}")
            print(f"Widget choice values: {widget.choice_values}")
            print(f"Widget display: {widget.field_display}")
            print(f"Widget bounding box: {widget.rect}")
            print('*' * 50)

        widgets_list.extend(l)

doc.close()


Widget type: 7
Widget field type string: Text
Widget label: SUSPECT ADVERSE REACTION REPORT
Widget name: SUSPECT ADVERSE REACTION REPORT
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(301.20001220703125, 91.85198974609375, 532.7999877929688, 114.4119873046875)
**************************************************
Widget type: 7
Widget field type string: Text
Widget label: 1. PATIENT INITIALS (first, last)
Widget name: 1 PATIENT INITIALS first last
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(60.720001220703125, 181.3709716796875, 144.47999572753906, 197.093994140625)
**************************************************
Widget type: 7
Widget field type string: Text
Widget label: 1a. COUNTRY
Widget name: 1a COUNTRY
Widget value: 
Widget choice values: None
Widget display: 0
Widget bounding box: Rect(146.63999938964844, 181.3709716796875, 219.1199951171875, 197.093994140625)
****************************************

In [66]:
import fitz
from PIL import Image
import pytesseract

# count = 1
for i in range(8):
    image_path = f"page_{i + 1}_bbox_image.png"
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
        print("="*50, i+1)
        print(text)

================================================== 1
YES (CL) N¢

================================================== 2
NO (CL) NA

================================================== 3
NA |

================================================== 4
S (CC) Ne

================================================== 5
NO ((_)NA

================================================== 6
NA) |

================================================== 7
JINITIAL (C

================================================== 8
)FOLLOWUP

LL



In [11]:
import pymupdf
# import os
import json
import io
import pytesseract
from PIL import Image

pdf_path = r"C:\Users\USER\Downloads\Fillable-Form_CIOMS-to-E2B.pdf"
doc = pymupdf.open(pdf_path)

dict_data = {}

json_file = open('dict_data.json', 'w')
for page in doc:
    widgets = page.widgets()
    if widgets:
        for widget in widgets:
            if widget.field_type_string == 'RadioButton':
                bounding_box = widget.rect
                increase_amount = 20
                new_bounding_box = pymupdf.Rect(bounding_box.x0 + 14, bounding_box.y0 + 3,
                                            bounding_box.x1 + increase_amount*2, bounding_box.y1)
                zoom_x = 9.0
                zoom_y = 8.0
                mat = pymupdf.Matrix(zoom_x, zoom_y)
                pix = page.get_pixmap(matrix=mat, clip=new_bounding_box)

                # Convert pixmap to PIL image
                img = Image.open(io.BytesIO(pix.tobytes("png")))


                text = pytesseract.image_to_string(img)
                required_text = text.split()[0]
                print(required_text)

                dict_data[f"{widget.field_name}_{required_text}"] = widget.field_name

                continue
            print(widget.field_name)
            dict_data[widget.field_name] = widget.field_name

json.dump(dict_data, json_file, indent=4)

json_file.close()

doc.close()


SUSPECT ADVERSE REACTION REPORT
1 PATIENT INITIALS first last
1a COUNTRY
2_DateOfBirth
2a AGE Years
3_Sex
4_ReactionOnset
7  13 DESCRIBE REACTIONS including relevant testslab data
PatientDied
InpatientHospitalisation
Persistence_Incapacity
LifeThreatening
14_SuspectDrugs
YES
NO
NA
15_DailyDoses
16_RoutesOfAdmin
17_IndicationsForUse
S
NO
NA
18_TherapyFrom
18_TherapyTo
19_TherapyDuration
22 CONCOMITANT DRUGS AND DATES OF ADMINISTRATION exclude those used to treat reaction
23 OTHER RELEVANT HISTORY eg diagnostics allergies pregnancy with last month of period etc
24A_NameOfManufacturer
24a  ADDRESS OF MANUFACTURER
24b MFR CONTROL NO
24_ManufacturerDetails
24c DATE RECEIVED BY MANUFACTURER
DATE OF THIS REPORT
24d_Study
24d_HealthProfessional
24d_Literature
JINITIAL
‘FOLLOWUP
